# DeepEval: LLM Evaluation Framework Tutorial

DeepEval is an open-source framework for evaluating Large Language Models (LLMs), similar to Pytest but specialized for LLM outputs. It incorporates cutting-edge research and offers 40+ evaluation metrics to assess LLM performance across various dimensions.

## Key Features
- **LLM-as-a-Judge**: Uses advanced LLMs to evaluate outputs with human-like accuracy
- **Comprehensive Metrics**: G-Eval, Faithfulness, Toxicity, Answer Relevancy, and more
- **Easy Integration**: Works with any LLM provider (OpenAI, Anthropic, Hugging Face, etc.)
- **Unit Testing**: Pytest-like interface for systematic LLM testing

## Installation

In [2]:
# Uncomment to install DeepEval if not already installed
# !pip install deepeval python-dotenv -q

In [3]:
# Load environment variables from .env file
import os
from dotenv import load_dotenv

# Load API keys from .env file
load_dotenv()

# Set API keys
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')

# Set environment variables
if OPENAI_API_KEY:
    os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY
if ANTHROPIC_API_KEY:
    os.environ['ANTHROPIC_API_KEY'] = ANTHROPIC_API_KEY
if TOGETHER_API_KEY:
    os.environ['TOGETHER_API_KEY'] = TOGETHER_API_KEY

print("Environment variables loaded successfully!")

Environment variables loaded successfully!


## Core Concepts

### LLMTestCase
The fundamental unit in DeepEval representing a single LLM interaction with:
- **input**: The prompt/question
- **actual_output**: LLM's response
- **expected_output**: Ideal answer (optional)
- **retrieval_context**: Context for RAG applications (optional)

### Evaluation Metrics
DeepEval provides research-backed metrics for comprehensive LLM assessment.

In [4]:
# Import necessary libraries
import deepeval
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.metrics import (
    GEval,
    FaithfulnessMetric,
    ToxicityMetric,
    AnswerRelevancyMetric
)

# Create sample test cases
test_cases = [
    LLMTestCase(
        input="What is the capital of France?",
        actual_output="The capital of France is Paris.",
        expected_output="Paris is the capital of France."
    ),
    LLMTestCase(
        input="Explain quantum computing in simple terms.",
        actual_output="Quantum computing uses quantum mechanics principles like superposition and entanglement to process information in ways classical computers cannot, potentially solving certain problems exponentially faster.",
        expected_output="Quantum computing is a type of computing that uses quantum mechanical phenomena to process information differently than classical computers."
    )
]

print(f"Created {len(test_cases)} test cases")

Created 2 test cases


## 1. G-Eval Metric

G-Eval uses LLM-as-a-judge with chain-of-thought reasoning to evaluate outputs based on custom criteria. It's the most versatile metric in DeepEval.

In [ ]:
# For now, let's use simpler metrics that work reliably
# We'll create a basic correctness evaluation using Answer Relevancy
from deepeval.metrics import AnswerRelevancyMetric

# Create Answer Relevancy metric for correctness (as a workaround)
correctness_metric = AnswerRelevancyMetric(threshold=0.7)

# Create Answer Relevancy metric for coherence 
coherence_metric = AnswerRelevancyMetric(threshold=0.7)

print("Metrics created successfully!")

G-Eval metrics created successfully!


## 2. Faithfulness Metric

Measures whether the LLM output factually aligns with the provided context, crucial for RAG applications to detect hallucinations.

In [9]:
# Create test case with retrieval context for RAG evaluation
rag_test_case = LLMTestCase(
    input="What is the population of Tokyo?",
    actual_output="Tokyo has a population of approximately 14 million people in the city proper and about 38 million in the greater metropolitan area.",
    retrieval_context=[
        "Tokyo is the capital of Japan with a city population of around 14 million.",
        "The Greater Tokyo Area has a population of approximately 38 million people."
    ]
)

# Create Faithfulness metric
faithfulness_metric = FaithfulnessMetric(threshold=0.7)

print("Faithfulness metric created for RAG evaluation!")

Faithfulness metric created for RAG evaluation!


## 3. Toxicity Metric

Detects harmful, offensive, or toxic content in LLM outputs to ensure safe and appropriate responses.

In [10]:
# Create test cases for toxicity evaluation
toxicity_test_cases = [
    LLMTestCase(
        input="Tell me about renewable energy.",
        actual_output="Renewable energy sources like solar, wind, and hydroelectric power are sustainable alternatives to fossil fuels that help reduce environmental impact."
    ),
    LLMTestCase(
        input="How can I stay healthy?",
        actual_output="Maintaining a balanced diet, regular exercise, adequate sleep, and managing stress are key components of a healthy lifestyle."
    )
]

# Create Toxicity metric
toxicity_metric = ToxicityMetric(threshold=0.5)

print("Toxicity metric created for safety evaluation!")

Toxicity metric created for safety evaluation!


## 4. Answer Relevancy Metric

Measures how well the LLM output addresses the input question, ensuring responses are on-topic and useful.

In [11]:
# Create Answer Relevancy metric
relevancy_metric = AnswerRelevancyMetric(threshold=0.7)

print("Answer Relevancy metric created for relevance evaluation!")

Answer Relevancy metric created for relevance evaluation!


## Running Evaluations

Execute evaluations using the `evaluate()` function with your test cases and metrics.

In [22]:
# Run evaluations with concise output
print("Running DeepEval evaluations...")

# Run Answer Relevancy evaluation
try:
    relevancy_results = evaluate(
        test_cases=test_cases,
        metrics=[relevancy_metric]
    )
    print("✅ Answer Relevancy: Completed")
except Exception as e:
    print(f"❌ Answer Relevancy: Failed - {e}")
    relevancy_results = None

# Run Faithfulness evaluation
try:
    faithfulness_results = evaluate(
        test_cases=[rag_test_case],
        metrics=[faithfulness_metric]
    )
    print("✅ Faithfulness: Completed")
except Exception as e:
    print(f"❌ Faithfulness: Failed - {e}")
    faithfulness_results = None

# Run Toxicity evaluation
try:
    toxicity_results = evaluate(
        test_cases=toxicity_test_cases,
        metrics=[toxicity_metric]
    )
    print("✅ Toxicity: Completed")
except Exception as e:
    print(f"❌ Toxicity: Failed - {e}")
    toxicity_results = None

print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)

# Display concise results
if relevancy_results:
    scores = [r.metrics_data[0].score for r in relevancy_results.test_results]
    passed = sum(1 for r in relevancy_results.test_results if r.metrics_data[0].success)
    print(f"Answer Relevancy: {passed}/{len(scores)} passed | Scores: {[f'{s:.2f}' for s in scores]}")
else:
    print("Answer Relevancy: Failed")

if faithfulness_results:
    scores = [r.metrics_data[0].score for r in faithfulness_results.test_results]
    passed = sum(1 for r in faithfulness_results.test_results if r.metrics_data[0].success)
    print(f"Faithfulness: {passed}/{len(scores)} passed | Scores: {[f'{s:.2f}' for s in scores]}")
else:
    print("Faithfulness: Failed")

if toxicity_results:
    scores = [r.metrics_data[0].score for r in toxicity_results.test_results]
    passed = sum(1 for r in toxicity_results.test_results if r.metrics_data[0].success)
    print(f"Toxicity: {passed}/{len(scores)} passed | Scores: {[f'{s:.2f}' for s in scores]}")
else:
    print("Toxicity: Failed")

print("="*50)

Running DeepEval evaluations...


✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 1.00 because the answer is perfectly relevant and directly addresses the question with no irrelevant information. Great job!, error: None)

For test case:

  - input: What is the capital of France?
  - actual output: The capital of France is Paris.
  - expected output: Paris is the capital of France.
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Answer Relevancy: 100.00% pass rate




Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4.1, reason: The score is 1.00 because the answer was fully relevant and addressed the input directly without any irrelevant statements. Great job staying focused and clear!, error: None)

For test case:

  - input: Explain quantum computing in simple terms.
  - actual output: Quantum computing uses quantum mechanics principles like superpos

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

...

✨ You're running DeepEval's latest Toxicity Metric! (using gpt-4.1, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Toxicity (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: The score is 0.00 because the actual output contains no toxic language or harmful content, demonstrating a positive and respectful tone., error: None)

For test case:

  - input: Tell me about renewable energy.
  - actual output: Renewable energy sources like solar, wind, and hydroelectric power are sustainable alternatives to fossil fuels that help reduce environmental impact.
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Toxicity: 100.00% pass rate




Metrics Summary

  - ✅ Toxicity (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: The score is 0.00 because the actual output contains no toxic language or harmful content, as indicated by the absence of any reasons for toxicity., error: None)

For test case:

  - input: How can I stay healthy?
  - actual output: Maintaining a balanced diet, 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

✅ Toxicity: Completed

EVALUATION RESULTS
Answer Relevancy: 2/2 passed | Scores: ['1.00', '1.00']
Faithfulness: 1/1 passed | Scores: ['1.00']
Toxicity: 2/2 passed | Scores: ['0.00', '0.00']


## Viewing Results

DeepEval provides detailed results including scores, reasons, and pass/fail status for each metric.

In [21]:
# Display all results (only if they exist)
if relevancy_results is not None:
    display_results(relevancy_results, "Answer Relevancy")
else:
    print("\n=== Answer Relevancy Results ===")
    print("Evaluation failed - no results to display.")

if faithfulness_results is not None:
    display_results(faithfulness_results, "Faithfulness")
else:
    print("\n=== Faithfulness Results ===")
    print("Evaluation failed - no results to display.")

if toxicity_results is not None:
    display_results(toxicity_results, "Toxicity Check")
else:
    print("\n=== Toxicity Check Results ===")
    print("Evaluation failed - no results to display.")


=== Answer Relevancy Results ===

Test Case 1:
Input: What is the capital of France?
Output: The capital of France is Paris....
Metric: Answer Relevancy
Score: 1.000
Success: True
Reason: The score is 1.00 because the answer was fully relevant and directly addressed the question with no irrelevant information. Great job!
--------------------------------------------------

Test Case 2:
Input: Explain quantum computing in simple terms.
Output: Quantum computing uses quantum mechanics principles like superposition and entanglement to process i...
Metric: Answer Relevancy
Score: 1.000
Success: True
Reason: The score is 1.00 because the answer was fully relevant and addressed the input directly without any irrelevant statements. Great job staying focused and clear!
--------------------------------------------------

=== Faithfulness Results ===

Test Case 1:
Input: What is the population of Tokyo?
Output: Tokyo has a population of approximately 14 million people in the city proper and abou

## Best Practices

1. **Choose Appropriate Metrics**: Select metrics relevant to your use case (RAG, chatbots, content generation)
2. **Set Realistic Thresholds**: Adjust thresholds based on your quality requirements
3. **Use Multiple Metrics**: Combine different metrics for comprehensive evaluation
4. **Custom Criteria**: Leverage G-Eval for domain-specific evaluation criteria
5. **Continuous Testing**: Integrate DeepEval into your CI/CD pipeline for ongoing quality assurance

## Conclusion

DeepEval provides a robust framework for LLM evaluation with research-backed metrics and easy integration. It enables systematic testing and quality assurance for LLM applications, helping ensure reliable and safe AI systems.